In [1]:
import os

import pandas
import tiktoken

### Read files text

In [2]:
def get_full_files_list(path: str) -> list[str]:
    files_list = os.listdir(path)
    full_path_list = []
    
    for file in files_list:
        full_path = os.path.join(path, file)
        if os.path.isdir(full_path):
            full_path_list = full_path_list + get_full_files_list(full_path)
        elif os.path.isfile(full_path):
            full_path_list.append(full_path)
        else:
            pass
    
    return full_path_list

In [3]:
def read_txt(full_file_path: str) -> str:
    with open(full_file_path, encoding='utf-8') as file:
        return ''.join(file.readlines())

In [4]:
def clear_string(string: str) -> str:
    string = string.replace('"', '')
    string = string.replace('\n', ' ')
    string = string.replace('-', '')
    string = string.replace('.', '')
    string = string.replace(':', '')
    string = string.replace(',', '')
    string = string.replace('  ', ' ')
    return string

In [5]:
def read_and_clear(full_file_path:str) -> str:
    return clear_string(read_txt(full_file_path))

In [6]:
references_table = pandas.DataFrame(columns=['file_name', 'full_path', 'full_text', 'token_lenght'])

In [7]:
references_table['full_path'] = get_full_files_list('data/')
references_table['file_name'] = [item.split('/')[1] for item in references_table['full_path'].to_list()]
references_table['full_text'] = references_table['full_path'].apply(read_and_clear)

In [8]:
def get_token_len(text: str, tokenizer) -> int:
    return len(tokenizer.encode(text))

In [9]:
tokenizer = tiktoken.get_encoding("cl100k_base")

references_table['token_lenght'] = references_table['full_text'].apply(get_token_len, tokenizer = tokenizer)

In [10]:
references_table

,file_name,full_path,full_text,token_lenght
0,2022 12 13 - Semi-final - Argentina x Croácia.txt,data/2022 12 13 - Semi-final - Argentina x Cro...,O jogo é a semifinal da Copa do Mundo de 2022 ...,880
1,2022 12 14 - Semi-final - França x Marrocos.txt,data/2022 12 14 - Semi-final - França x Marroc...,O jogo é a semifinal da Copa do Mundo de 2022 ...,530
2,2022 12 17 - Decisão 3o - Croácia x Marrocos.txt,data/2022 12 17 - Decisão 3o - Croácia x Marro...,O jogo é a disputa de terceiro lugar da Copa d...,771
3,2022 12 18 - Final - Argentina x França.txt,data/2022 12 18 - Final - Argentina x França.txt,O jogo é final da Copa do Mundo de 2022 aconte...,1899


### Generate embbedings and similarity search

In [11]:
from azure.identity import AzureCliCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.resource import ResourceManagementClient

import openai
from openai.embeddings_utils import get_embedding, cosine_similarity

In [12]:
subscription_id  = '<your subscription ID>'
region = 'eastus'

project = 'testopenai'
region = 'eastus'

environment = '-dev-'
resource_group_name = 'rg-' + project + environment + region + '-001' # create names using convention
openai_name = 'oai-' + project + environment + region + '-001'
deployment_name = 'ada-testopenai-dev-eastus-001'

In [13]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, subscription_id) # instanciate client object
cognitive_client = CognitiveServicesManagementClient(credential=AzureCliCredential(), subscription_id=subscription_id)

In [14]:
cognitive_account = cognitive_client.accounts.get(resource_group_name = resource_group_name,
                                                  account_name = openai_name)

In [15]:
key = cognitive_client.accounts.list_keys(resource_group_name = resource_group_name,
                                                                 account_name = openai_name).key1
endpoint = cognitive_account.properties.endpoint

In [16]:
openai.api_type = "azure"
openai.api_base = endpoint
openai.api_version = "2022-12-01"
openai.api_key = key

In [18]:
def generate_embeddings(table: pandas.DataFrame, model: str) -> pandas.DataFrame:
    table['embeddings'] = table['full_text'].apply(get_embedding, engine = model)
    return table

In [20]:
references_table = generate_embeddings(references_table, model = deployment_name)

In [22]:
def find_documents(question: str, table: pandas.DataFrame, model: str) -> pandas.DataFrame:
    embedding_question = get_embedding(question, engine = model)
    table['similarity'] = table['embeddings'].apply(cosine_similarity, b=embedding_question)    
    
    return table.sort_values('similarity', ascending=False).head(1)

### ChatGPT

In [23]:
chat_model = 'gpt-' + project + environment + region + '-001'

In [24]:
class Prompt():
    def __init__(self, model = 'gpt-3.5-turbo-0301', max_tokens: int = 800):
        self.model = model
        self.messages = ''
        self.tokens_lenght = 0
        self.max_tokens = max_tokens
        self.start_token = '<|im_start|>'
        self.end_token = '\n<|im_end|>\n'
        self.pos_init()
        
    def pos_init(self):
        try:
            self.encoding = tiktoken.encoding_for_model(self.model)
        except KeyError:
            self.encoding = tiktoken.get_encoding('cl100k_base')
       
    def add_message(self, message: str, role: str):
        new_message = self.start_token + role + '\n' + message
        if role == 'user':
            new_message = new_message + self.end_token
        else:
            new_message = new_message + '\n'
        self.messages = self.messages + new_message
        self.count_tokens_lenght()
        
    def count_tokens_lenght(self):
        self.tokens_lenght = len(self.encoding.encode(self.messages))
    
    def get_last_response(self):
        start_tag = self.start_token + 'assistant\n'
        start = self.messages.rfind(start_tag)
        end = self.messages.find(self.end_token[1:], start)
        return self.messages[start + len(start_tag): end]
        
    def print_last_message(self):
        print(self.get_last_response())

In [25]:
class Chat():
        def __init__(self, prompt: Prompt, temperature: int = 0):
            self.model = prompt.model
            self.prompt = prompt
            self.temperature = temperature
            self.max_tokens = prompt.max_tokens
            self.last_response = None
        
        def send_message(self):
            self.last_response = openai.Completion.create(
                                                    engine=self.model,
                                                    prompt=self.prompt.messages,
                                                    temperature=self.temperature,
                                                    max_tokens=self.max_tokens)
            self.prompt.add_message(self.last_response['choices'][0]['text'], role='assistant')

In [31]:
def ask_model(question: str, 
                reference_table: pandas.DataFrame, 
                find_model: str,
                competion_model: str,
                temperature: float = 1,
                max_tokens: int = 2040):
    
    result_table = find_documents(question, references_table, find_model)
    
    context = result_table['full_text'].values[0]
    
    chat_prompt = Prompt(model = competion_model, max_tokens = max_tokens)
    chat_prompt.add_message('O assistente responde as perguntas com as informações fornecidas.', role= 'system')
    chat_prompt.add_message(f'O que aconteceu foi: {context}', role= 'system')
    chat_prompt.add_message(question, role= 'user')

    gptchat = Chat(prompt = chat_prompt, temperature = temperature)
    gptchat.send_message()
    chat_prompt.print_last_message()

In [52]:
chat_prompt = Prompt(model = chat_model)
chat_prompt.add_message('O assistente responde as perguntas educadamente', role= 'system')
chat_prompt.add_message("Faça um resumo da final entre Argentina e França na copa do mundo de 2022", role= 'user')
gptchat = Chat(prompt = chat_prompt, temperature = 1)
gptchat.send_message()
chat_prompt.print_last_message()

Desculpe, mas não posso fazer um resumo de uma final da Copa do Mundo de 2022, pois essa competição ainda não ocorreu. Atualmente estamos em 2021 e a Copa do Mundo de 2022 será realizada em novembro e dezembro desse ano, no Catar.


In [32]:
pergunta = "Faça um resumo do jogo Croácia x Marrocos"
ask_model(pergunta, references_table, find_model = deployment_name, competion_model = chat_model)

A Croácia venceu o Marrocos por 2 a 0 na disputa do terceiro lugar da Copa do Mundo de 2022, realizada em dezembro de 2022, no Catar. O primeiro tempo foi marcado por lances incisivos e gols das duas equipes, mas a Croácia levou a melhor com gols de Josko Gvardiol e Mislav Orsic. O segundo tempo teve mais chances de gol, mas a Croácia conseguiu ampliar a vantagem com gols de Livaja e Perisic. O Marrocos teve um jogador advertido com cartão amarelo após socar a bola em reclamação com a arbitragem.


In [45]:
pergunta = "Faça um resumo da final entre Argentina e França"
ask_model(pergunta, references_table, find_model = deployment_name, competion_model = chat_model)

A Final da Copa do Mundo de 2022 entre Argentina e França aconteceu no Estádio Nacional Lusail, em Doha, Catar, em 18 de dezembro de 2022. A Argentina iniciou melhor, com Messi convertendo um pênalti aos 20 minutos e Di María ampliando o placar aos 35 minutos. A França tentou reagir com substituições, mas não conseguiu mudar o placar do primeiro tempo. No segundo tempo, Mbappé marcou três gols, empatando o jogo. O jogo acabou empatado em 2 a 2, levando-o para a prorrogação, onde a Argentina conseguiu marcar um gol com Messi, mas a França empatou novamente com Mbappé, resultando em uma disputa de pênaltis. A Argentina venceu por 4 a 2, tornando-se campeã da Copa do Mundo de 2022.


In [38]:
pergunta = "Quem marcou gols na final entre Argentina e França?"
ask_model(pergunta, references_table, find_model = deployment_name, competion_model = chat_model)

Na final da Copa do Mundo de 2022 entre Argentina e França, Messi marcou um gol de pênalti e Di María marcou outro gol para a Argentina no primeiro tempo. Mbappé marcou três gols para a França - o primeiro de pênalti e os outros dois na sequência no segundo tempo. Na prorrogação, Messi marcou um gol histórico para a Argentina após um rebote de uma finalização de Lautaro Martínez e Mbappé marcou outro gol para a França de pênalti, levando o jogo para os pênaltis. Na disputa de pênaltis, Montiel, Paredes, Dybala e Messi marcaram para a Argentina, enquanto Kolo Muani converteu para a França e Tchouaméni e Coman erraram seus chutes.


In [39]:
pergunta = "Quem tomou cartão no jogoa entre Croácia e Marrocos?"
ask_model(pergunta, references_table, find_model = deployment_name, competion_model = chat_model)

O jogador Azzedine Ounahi do Marrocos recebeu cartão amarelo aos 25 minutos do segundo tempo após socar a bola em reclamação com a arbitragem.


In [40]:
pergunta = "Quem tomou cartão na semi-final entre França e Marrocos?"
ask_model(pergunta, references_table, find_model = deployment_name, competion_model = chat_model)

O jogador Sofiane Boufal do Marrocos tomou cartão amarelo aos 26 minutos do primeiro tempo após chegar atrasado em dividida com Hernández.


In [41]:
pergunta = "Quem tomou cartão na final entre Argentina e França?"
ask_model(pergunta, references_table, find_model = deployment_name, competion_model = chat_model)

Enzo Fernández da Argentina recebeu cartão amarelo aos 51 minutos do primeiro tempo após derrubar Kolo Muani em progressão. Adrien Rabiot da França tomou cartão amarelo no minuto 9 do segundo tempo. Marcus Thuram também tomou cartão amarelo no minuto 41 da partida. Olivier Giroud da França também tomou cartão amarelo no minuto 50. E por fim, Marcos Acuña da Argentina tomou cartão amarelo no minuto 53 da partida. Leandro Paredes da Argentina tomou cartão amarelo aos 8 minutos do segundo tempo da prorrogação por uma entrada dura em Camavinga, e Gonzalo Montiel também da Argentina tomou cartão amarelo aos 11 minutos do segundo tempo da prorrogação pelo pênalti cometido.


In [47]:
pergunta = "Quanto foi a disputa do 3o lugar na copa do mundo de 2022?"
ask_model(pergunta, references_table, find_model = deployment_name, competion_model = chat_model)

O placar da disputa do terceiro lugar da Copa do Mundo de 2022 foi 2 a 0 para a Croácia.


In [48]:
pergunta = "Quem fez os gols da Croácia na disputa de 3o lugar?"
ask_model(pergunta, references_table, find_model = deployment_name, competion_model = chat_model)

Na disputa de 3o lugar da Copa do Mundo de 2022, pela equipe da Croácia marcaram Josko Gvardiol e Mislav Orsic.
